# Display of List of Available Widgets to Display 

## Overview 

The visualizer is planned to support three visualization modes:
1. Visualize updates to a standalone kvlang script
2. Visualize updates to an existing kivy widget
3. Visualize updates to an existing kivy application 

To allow the user to choose what to visualize, we should populate a listbox that displays each of the visualization options. The user will be able to select which item they would like to visualize. Switching between items will update the visualization. 

## Research

### How do you search a py file for class definitions?
We need to find all App and Widget descendants within each python file. We cannot expect all widget types to be a direct descendant of known kivy types. 

The `inspect` python module will return all of the base classes of a type, but this module requires live objects. We want to avoid executing the user's code, because this will require using their environment and we are trying to keep a clear separation between their environment and the kivy designer environment - plus actually executing their code can have some side effects. We should only do this execution during visualization, when they request it.

Unfortunately we cannot we use the `ast` module to find all objects within a python file that descend from the `App` and `widget` class. The reason is that the AST tree only stores the direct class parents, and does not provide any reference to where those parents are declared. To build a more complete hierarchy tree we would need to search through all the available source files and build the hierarchy ourselves. 


In [ ]:
import ast

# This code fails because ast does not store detailed information
# about ClassDef parents

def get_subclasses_of_widget(filename):
    subclasses = []
    with open(filename, "r") as f:
        tree = ast.parse(f.read())
        for node in ast.iter_child_nodes(tree):
            if isinstance(node, ast.ClassDef):
                if is_derived_from_widget(node):
                    subclasses.append(node.name)
    return subclasses

def is_derived_from_widget(node):
    for base in node.bases:
        if isinstance(base, ast.Name) and base.id == "Widget":
            return True
        elif is_derived_from_widget(base): #ERROR. Can't recursively traverse the tree this way. Base has no attr bases.
            return True
    return False

subclasses = get_subclasses_of_widget("C:\\Users\\joshu\\source\\repos\\kivydesigner\\kivydesigner\\uix\\kdfilechooser.py")
print(subclasses)


### What entries should the listbox contain?

(In this order)
1. Names of custom user defined applications
2. Names of custom user defined kvlang scripts
3. Names of custom user defined widgets
4. Names of kivy standard library widgets

In the future we should also populate the listbox with kivy garden widgets. 

### How should the entries be displayed?

A treeview would be appropriate here, since this will allow us to categorize the widgets into groups. 

Unfortunately, we probably won't be able to reuse some of the styling work we did on the kdfileviewer since the file viewer has a complex interplay of objects - but we can apply the same learnings here at the expense of some duplication. 

### How are custom attribute types declared?

To-Do here

## General Solution Plan



1. Add a WidgetScanner class that can recursively search a file directory for `Widget` and `App` classes. The search should avoid compiling any code, so our plan is to use the abstract syntax tree
    * The WidgetScanner class should accept the root file directory
    * It should return the module filepath for each class type
    * It should provide the four groups of widget types specified in the research section (kvfiles, user Apps, user widgets, kivy widgets)
    * It should provide a mechanism to rescan file(s) on update, without rescanning the entire directory
2. Use the WidgetScanner class to find all of the `Widget` and `App` within the `kivy` standard library (for the current version for now), and store these class names in a list
3. Use the list of kivy standard library classes to improve the search within WidgetScanner. The WidgetScanner doesn't need to search up to the widget. It can stop at any of the standard library classes
4. Our search will not be perfect since the ast does not provide detailed information about the parents. Since the ast does not state where the parent class is declared, our imperfect search will have a very small probability of returning false positive (if there is another class in another namespace with the same name as a kivy standard library widget), and we risk missing widgets if the widget descends from a third-party widget that is not present in the available source files. To account for these rare failures, we should provide attributes to explicitely enable and disable the visualization (@disable_kv_vis, @enable_kv_vis)
5. Use a tree view to display the four groups
  * Let's stylize the tree in a separate issue. I'd like to wait until the visualizer is able to help us here 
6. Add a on_submit event handler that will send the proper update instruction to the visualizer whenever an item is selected. Create some dummy instructions for now since hot reloading for class types is not yet implemented

## Implementation Questions

### How should `WidgetScanner` search a directory for widgets and apps?

Write a function that returns (app, widget, kvfile) lists.

Make sure the search uses the new attributes


### How should `WidgetScanner` store the widget types along with the filepath, to allow file-by-file update?




In [14]:
import ast 
from pathlib import Path

'''
        self.maybe_newline()
        for deco in node.decorator_list:
            self.fill("@")
            self.traverse(deco)
        self.fill("class " + node.name)
        with self.delimit_if("(", ")", condition = node.bases or node.keywords):
            comma = False
            for e in node.bases:
                if comma:
                    self.write(", ")
                else:
                    comma = True
                self.traverse(e)
            for e in node.keywords:
                if comma:
                    self.write(", ")
                else:
                    comma = True
                self.traverse(e)
'''
class bb:
    pass

class cc:
    pass 

class aa(bb, cc):
    pass 

class dd(aa, bb):
    pass 

class ee(aa):
    pass 



class ClassDefVisitor(ast.NodeVisitor):

    def visit_ClassDef(self, node):
        print(f'{node.name}: {[nd.id for nd in node.bases]}')
        for base_node in node.bases:
            self.visit(ast.Call(func=base_node))

EX_FILE = "C:\\Users\\joshu\\source\\repos\\kivydesigner\\kivydesigner\\uix\\kdfilechooser.py"
b  = Path(EX_FILE).read_text()
tree = ast.parse(b)
tree = ast.parse("class A: pass\nclass B(A): pass\nclass C(B): pass")
ClassDefVisitor().visit(tree)


A: []
B: ['A']
C: ['B']
